In [208]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import time
import re
import warnings
warnings.filterwarnings('ignore')
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import json
from datetime import datetime, timedelta
import os
chromedriver = "/Users/stevenwang/Projects/funded-and-hiring-site/newsletter_scraper/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
import pyperclip
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pytest_timeout
import dateparser

In [209]:
def get_content(url):
    page = requests.get(url)
    return page.content

In [210]:
ttnews_content = get_content('https://www.ttnews.com/articles')

In [211]:
fw_news_content = get_content('https://www.freightwaves.com/news/category/trucking')

In [212]:
overdrive_business_content = get_content('https://www.overdriveonline.com/business')

In [213]:
samsara_news_content = get_content('https://www.samsara.com/company/news')

In [214]:
platform_science_news_content = get_content('https://www.platformscience.com/in-the-news')

In [215]:
omnitracs_news_content = get_content('https://www.omnitracs.com/company/news')

In [216]:
geotab_news_content = get_content('https://www.geotab.com/press-room/')

In [217]:
def get_ttnews(content):
    soup = BeautifulSoup(content, "html.parser")

    companies = []
    description = []
    publish_date = []
    company_urls = []
    stories = []
    titles = []
    images = []
    
    articles = soup.find_all(class_='views-row')
    
    for article in articles:
        title = article.find('a')
        date = article.find(class_='date-display-single')
        text =  article.find('p') # description
        image = article.find('img')

        if (date != None and title != None and text != None):
            parsed_date = dateparser.parse(date['content'])
            companies.append('TTNews')
            description.append(text.get_text().strip())
            publish_date.append(parsed_date.date())
            stories.append('https://ttnews.com' + title.get('href'))
            titles.append(title.getText().strip())
            images.append(image.get('src'))
            company_urls.append('https://ttnews.com/')

    dataframe = pd.DataFrame({'company':companies, 'headline':titles, 'description':description, 'publish_date':publish_date, 'story_url':stories, 'company_url':company_urls, 'image':images})
    return dataframe

ttnews = get_ttnews(ttnews_content)

ttnews

,company,headline,description,publish_date,story_url,company_url,image
0,TTNews,Pelosi Regroups on Infrastructure With Hopes f...,House Speaker Nancy Pelosi (D-Calif.) plans to...,2021-10-01,https://ttnews.com/articles/pelosi-regroups-in...,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
1,TTNews,Daimler Investors Approve Plan to Split Merced...,Daimler AG shareholders backed a plan to spin ...,2021-10-01,https://ttnews.com/articles/daimler-investors-...,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
2,TTNews,"Nikola, Opal Fuels to Build Hydrogen Fueling S...",Nikola Corp. reached a preliminary deal with O...,2021-10-01,https://ttnews.com/articles/nikola-opal-fuels-...,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
3,TTNews,"No Vote Held on Infrastructure, but Talks to R...",WASHINGTON — Despite a long night of frantic n...,2021-10-01,https://ttnews.com/articles/no-vote-held-infra...,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
4,TTNews,TT Newsmakers,"Sgt. Zach Heard, Illinois State Police, and Ke...",2021-09-30,https://ttnews.com/newsmakers,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
5,TTNews,ATRI Report Urges Electric Tax to Help Fund Hi...,A new report from the American Transportation ...,2021-09-30,https://ttnews.com/articles/atri-report-urges-...,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
6,TTNews,Transportation Stakeholders Press Congress on ...,To guarantee long-term funding for the country...,2021-09-30,https://ttnews.com/articles/transportation-sta...,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
7,TTNews,Congress Clears Government Funding Bill to Ave...,The House passed a nine-week spending bill to ...,2021-09-30,https://ttnews.com/articles/senate-passes-fund...,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
8,TTNews,Transport Topics Radio,Transport Topics Radio is a weekly news roundt...,2021-09-30,https://ttnews.com/transport-topics-radio,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
9,TTNews,Port Leaders Warn of Supply Chain Challenges a...,"AUSTIN, Texas — The leader of one of the natio...",2021-09-30,https://ttnews.com/articles/port-leaders-warn-...,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...


In [218]:
def get_overdrive_business(content):
    soup = BeautifulSoup(content, "html.parser")

    companies = []
    description = []
    publish_date = []
    company_urls = []
    stories = []
    titles = []
    images = []
    
    articles = soup.find_all(class_='node-list__node')
    # print(articles)
    for article in articles:
        title = article.find('h5', class_='section-feed-content-node__content-short-name').a
        # print(title)
        date = article.find(class_='section-feed-content-node__content-published')
        # print(date)
        text =  article.find(class_='section-feed-content-node__content-teaser').a
        # print(text)
        image = article.find('img')

        if (date != None and title != None and text != None):
            parsed_date = dateparser.parse(date.get_text())
            companies.append('Overdrive')
            description.append(text.get_text().strip())
            publish_date.append(parsed_date.date())
            stories.append('https://www.overdriveonline.com' + title.get('href'))
            titles.append(title.getText().strip())
            images.append(image['src'])
            company_urls.append('https://www.overdriveonline.com/')

    dataframe = pd.DataFrame({'company':companies, 'headline':titles, 'description':description, 'publish_date':publish_date, 'story_url':stories, 'company_url':company_urls, 'image':images})
    return dataframe

overdrive_business = get_overdrive_business(overdrive_business_content)

overdrive_business

,company,headline,description,publish_date,story_url,company_url,image
0,Overdrive,An edge on growth: One way small fleets stand ...,Could Biden's vax mandate enhance the competit...,2021-09-30,https://www.overdriveonline.com/channel-19/art...,https://www.overdriveonline.com/,https://img.overdriveonline.com/files/base/ran...
1,Overdrive,Landstar owner-op poised for growth after winn...,Texas-based hauler Mike Lamb said winning the ...,2021-09-30,https://www.overdriveonline.com/life/article/1...,https://www.overdriveonline.com/,https://img.overdriveonline.com/files/base/ran...
2,Overdrive,Bryan Smith bags the big prize from TCA/Overdrive,The Owner-Operator of the Year was named Tuesd...,2021-09-29,https://www.overdriveonline.com/owner-operator...,https://www.overdriveonline.com/,https://img.overdriveonline.com/files/base/ran...
3,Overdrive,Partners in Business tip: Work to improve your...,Do not wait around and accumulate more debt ov...,2020-12-02,https://www.overdriveonline.com/partners-in-bu...,https://www.overdriveonline.com/,"data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK..."
4,Overdrive,Fuel prices surge to nearly 7-year high,The modest price increase over the past week b...,2021-09-28,https://www.overdriveonline.com/business/artic...,https://www.overdriveonline.com/,"data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK..."
5,Overdrive,Are owner-operators a legal landmine for fleet...,Recent court rulings cast doubt on the future ...,2021-09-28,https://www.overdriveonline.com/home/article/1...,https://www.overdriveonline.com/,"data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK..."
6,Overdrive,FMCSA shuts down New York-based trucker after ...,Bobir M. Kholmurodov has been effectively shut...,2021-09-27,https://www.overdriveonline.com/regulations/ar...,https://www.overdriveonline.com/,"data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK..."
7,Overdrive,Breaking Free: The history of the owner-operat...,From fighting for freedom to haul in its early...,2011-09-01,https://www.overdriveonline.com/business/artic...,https://www.overdriveonline.com/,"data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK..."
8,Overdrive,Tractor-trailers banned from section of I-95; ...,DelDot urges truckers to use I-495 as an alter...,2021-09-23,https://www.overdriveonline.com/home/article/1...,https://www.overdriveonline.com/,"data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK..."
9,Overdrive,DOT asks for truckers' input on supply chain d...,DOT will use information from the trucking ind...,2021-09-22,https://www.overdriveonline.com/regulations/ar...,https://www.overdriveonline.com/,"data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK..."


In [219]:
def get_samsara_news(content):
    soup = BeautifulSoup(content, "html.parser")

    companies = []
    description = []
    publish_date = []
    company_urls = []
    stories = []
    titles = []
    images = []
    
    articles = soup.find_all(class_='row')
    # print(articles)
    for article in articles:
        title = article.find(class_='source-quote')
        link = article.find('a')
        # print(title)
        date = article.find(class_='source-date')
        # print(date)
        text =  article.find(class_='source-quote')
        # print(text)
        image = article.find('img')
        # print(image)

        if (link != None and date != None and title != None and text != None):
            parsed_date = dateparser.parse(date.get_text())
            companies.append('Samsara')
            description.append(text.get_text().strip().strip('\"'))
            publish_date.append(parsed_date.date())
            stories.append(link.get('href'))
            titles.append(title.getText().strip().strip('\"'))
            
            if (image != None and image.get('src')):
                images.append(image['src'])
            elif (image != None and image.get('data-lazy')):
                images.append(image['data-lazy'])
            else:
                images.append(None)
            
            company_urls.append('https://www.samsara.com/')

    dataframe = pd.DataFrame({'company':companies, 'headline':titles, 'description':description, 'publish_date':publish_date, 'story_url':stories, 'company_url':company_urls, 'image':images})
    return dataframe

samsara_news = get_samsara_news(samsara_news_content)

samsara_news

,company,headline,description,publish_date,story_url,company_url,image
0,Samsara,Samsara Ranked on LinkedIn's 2021 Top Startups...,Samsara Ranked on LinkedIn's 2021 Top Startups...,2021-09-22,https://www.prnewswire.com/news-releases/samsa...,https://www.samsara.com/,//images.ctfassets.net/bx9krvy0u3sx/6Ey6tmvh1M...
1,Samsara,The State of Tennessee has awarded Samsara a s...,The State of Tennessee has awarded Samsara a s...,2021-09-17,https://www.government-fleet.com/10151760/tenn...,https://www.samsara.com/,//images.ctfassets.net/bx9krvy0u3sx/47F6OkjxOk...
2,Samsara,Samsara Selected as Connected Operations Platf...,Samsara Selected as Connected Operations Platf...,2021-09-13,https://www.prnewswire.com/news-releases/samsa...,https://www.samsara.com/,//images.ctfassets.net/bx9krvy0u3sx/2o8KuZK17L...
3,Samsara,Samsara Announces Confidential Submission of D...,Samsara Announces Confidential Submission of D...,2021-09-01,https://www.prnewswire.com/news-releases/samsa...,https://www.samsara.com/,//images.ctfassets.net/bx9krvy0u3sx/66jIszSDer...
4,Samsara,Ethical AI is still in its nascency but is bec...,Ethical AI is still in its nascency but is bec...,2021-08-29,https://venturebeat.com/2021/08/29/how-to-make...,https://www.samsara.com/,//images.ctfassets.net/bx9krvy0u3sx/2UpvaHxdNk...
...,...,...,...,...,...,...,...
95,Samsara,Some telling data points come this week from S...,Some telling data points come this week from S...,2020-06-16,https://mashable.com/article/speeding-crashes-...,https://www.samsara.com/,//images.ctfassets.net/bx9krvy0u3sx/7FJJ0zDU2t...
96,Samsara,"While speeds were rising, incidents of harsh b...","While speeds were rising, incidents of harsh b...",2020-06-16,https://finance.yahoo.com/news/truck-traffic-n...,https://www.samsara.com/,//images.ctfassets.net/bx9krvy0u3sx/5nqMKhHlvz...
97,Samsara,"To help maintain social distancing protocols, ...","To help maintain social distancing protocols, ...",2020-06-15,https://www.automotive-fleet.com/359959/8-ways...,https://www.samsara.com/,//images.ctfassets.net/bx9krvy0u3sx/2khYrATrnU...
98,Samsara,"The newest member of Together for Safer Roads,...","The newest member of Together for Safer Roads,...",2020-06-11,https://www.freightwaves.com/news/commentary-i...,https://www.samsara.com/,//images.ctfassets.net/bx9krvy0u3sx/5NgX9gwLrd...


In [220]:
def get_platform_science_news(content):
    soup = BeautifulSoup(content, "html.parser")

    companies = []
    description = []
    publish_date = []
    company_urls = []
    stories = []
    titles = []
    images = []
    
    articles = soup.find_all(class_='media-item')
    # print(articles)
    for article in articles:
        title = article.find(class_='media-heading')
        link = article.find('a')
        # print(title)
        date = article.find(class_='media-date-text')
        # print(date)
        text =  article.find(class_='media-heading')
        # print(text)
        image = article.find(class_='media-image-wrapper')
        # print(image)

        if (link != None and date != None and title != None and text != None):
            parsed_date = dateparser.parse(date.get_text())
            companies.append('Platform Science')
            description.append(text.get_text().strip())
            publish_date.append(parsed_date.date())
            stories.append(link.get('href'))
            titles.append(title.getText().strip())
            
            # <div style="background-image:url(&quot;https://global-uploads.webflow.com/606b95eef43b7bb577930a6d/6148b7d0d8590b7777d1d116_PS_Stevens.jpeg&quot;)" class="media-image-wrapper"></div>
            if (image != None and image.get('style')):
                images.append(image['style'].replace('background-image:url("', '').replace('")', ''))
            else:
                images.append(None)
            
            company_urls.append('https://www.platformscience.com/')

    dataframe = pd.DataFrame({'company':companies, 'headline':titles, 'description':description, 'publish_date':publish_date, 'story_url':stories, 'company_url':company_urls, 'image':images})
    return dataframe

platform_science_news = get_platform_science_news(platform_science_news_content)

platform_science_news

,company,headline,description,publish_date,story_url,company_url,image
0,Platform Science,Stevens partners with Platform Science,Stevens partners with Platform Science,2021-09-17,https://www.refrigeratedtransporter.com/carrie...,https://www.platformscience.com/,https://global-uploads.webflow.com/606b95eef43...
1,Platform Science,Automated 'e-Inspections' demonstrated at CVSA...,Automated 'e-Inspections' demonstrated at CVSA...,2021-09-02,https://www.overdriveonline.com/business/artic...,https://www.platformscience.com/,https://global-uploads.webflow.com/606b95eef43...
2,Platform Science,Platform Science partners with E-SMART to impr...,Platform Science partners with E-SMART to impr...,2021-08-26,https://ajot.com/news/article/platform-science...,https://www.platformscience.com/,https://global-uploads.webflow.com/606b95eef43...
3,Platform Science,Three states automating Level 3 inspections at...,Three states automating Level 3 inspections at...,2021-07-05,https://www.ccjdigital.com/technology/article/...,https://www.platformscience.com/,https://global-uploads.webflow.com/606b95eef43...
4,Platform Science,Drivewyze Opens Enrollment for e-Inspections a...,Drivewyze Opens Enrollment for e-Inspections a...,2021-06-29,https://www.truckinginfo.com/10146487/drivewyz...,https://www.platformscience.com/,https://global-uploads.webflow.com/606b95eef43...
...,...,...,...,...,...,...,...
91,Platform Science,FreightTech 25 and Platform Science Consolidat...,FreightTech 25 and Platform Science Consolidat...,2018-11-14,https://bit.ly/2FUEeLz,https://www.platformscience.com/,https://global-uploads.webflow.com/606b95eef43...
92,Platform Science,MarketWaves18’s Demo Day Best in Show Announced,MarketWaves18’s Demo Day Best in Show Announced,2018-11-14,https://bit.ly/2S9m8LD,https://www.platformscience.com/,https://global-uploads.webflow.com/606b95eef43...
93,Platform Science,Platform Science Integrates with Drivewyze Byp...,Platform Science Integrates with Drivewyze Byp...,2018-08-16,https://bit.ly/2FXwdpc,https://www.platformscience.com/,https://global-uploads.webflow.com/606b95eef43...
94,Platform Science,"Schneider, 8VC Partner to Boost Digital Supply...","Schneider, 8VC Partner to Boost Digital Supply...",2018-04-11,https://www.ttnews.com/articles/schneider-8vc-...,https://www.platformscience.com/,https://global-uploads.webflow.com/606b95eef43...


In [221]:
def get_omnitracs_news(content):
    soup = BeautifulSoup(content, "html.parser")

    companies = []
    description = []
    publish_date = []
    company_urls = []
    stories = []
    titles = []
    images = []
    
    articles = soup.find_all(class_='views-row')
    # print(articles)
    for article in articles:
        title = article.find(class_='field-title')
        link = article.find('article')
        # print(title)
        date = article.find(class_='field--name-field-date')
        # print(date)
        text =  article.find(class_='field-title')
        # print(text)

        if (link != None and date != None and title != None and text != None):
            parsed_date = dateparser.parse(date.get_text())
            companies.append('Omnitracs')
            description.append(text.get_text().strip())
            publish_date.append(parsed_date.date())
            stories.append(link.get('about'))
            titles.append(title.getText().strip())
            
            images.append(None)
            
            company_urls.append('https://www.omnitracs.com/')

    dataframe = pd.DataFrame({'company':companies, 'headline':titles, 'description':description, 'publish_date':publish_date, 'story_url':stories, 'company_url':company_urls, 'image':images})
    return dataframe

omnitracs_news = get_omnitracs_news(omnitracs_news_content)

omnitracs_news

,company,headline,description,publish_date,story_url,company_url,image
0,Omnitracs,Omnitracs Latest ELD Supplier to Be Comfortabl...,Omnitracs Latest ELD Supplier to Be Comfortabl...,2021-02-12,http://cantruck.ca/omnitracs-latest-eld-supplier/,https://www.omnitracs.com/,None
1,Omnitracs,Primed for growth: Liquid Trucking turns misfo...,Primed for growth: Liquid Trucking turns misfo...,2021-01-18,https://www.ccjdigital.com/technology/article/...,https://www.omnitracs.com/,None
2,Omnitracs,Will Technology Convergence Streamline Fleet M...,Will Technology Convergence Streamline Fleet M...,2021-01-07,https://roadsigns.ttnews.com/episodes/will-tec...,https://www.omnitracs.com/,None
3,Omnitracs,What you should know about the looming Canadia...,What you should know about the looming Canadia...,2020-12-17,https://www.overdriveonline.com/channel-19/art...,https://www.omnitracs.com/,None
4,Omnitracs,SmartDrive Wins 2020 AutoTech Breakthrough Awa...,SmartDrive Wins 2020 AutoTech Breakthrough Awa...,2020-11-17,/news/smartdrive-wins-2020-autotech-breakthrou...,https://www.omnitracs.com/,None
5,Omnitracs,J&M Tank Lines Deploys SmartDrive Video-Based ...,J&M Tank Lines Deploys SmartDrive Video-Based ...,2020-10-29,/news/jm-tank-lines-deploys-smartdrive-video-b...,https://www.omnitracs.com/,None
6,Omnitracs,Southern Tank Transport Realizes Immediate Saf...,Southern Tank Transport Realizes Immediate Saf...,2020-09-16,/news/southern-tank-transport-realizes-immedia...,https://www.omnitracs.com/,None
7,Omnitracs,Next Generation of Wireless Technology: 5G Hol...,Next Generation of Wireless Technology: 5G Hol...,2020-09-09,https://www.ttnews.com/articles/next-generatio...,https://www.omnitracs.com/,None
8,Omnitracs,Keeping ELDs Up and Running,Keeping ELDs Up and Running,2020-09-09,https://www.ttnews.com/articles/keeping-elds-a...,https://www.omnitracs.com/,None
9,Omnitracs,RATP Dev USA Renews SmartDrive After 25% Drop ...,RATP Dev USA Renews SmartDrive After 25% Drop ...,2020-08-25,/news/ratp-dev-usa-renews-smartdrive-after-25-...,https://www.omnitracs.com/,None


In [222]:
def get_geotab_news(content):
    soup = BeautifulSoup(content, "html.parser")

    companies = []
    description = []
    publish_date = []
    company_urls = []
    stories = []
    titles = []
    images = []
    
    articles = soup.find_all(class_='MuiGrid-item')
    # print(articles)
    for article in articles:
        title = article.find(class_='MuiTypography-subtitle1')
        link = article.find('a')
        # print(title)
        date = article.find(class_='MuiTypography-body2')
        # print(date)
        text =  article.find(class_='MuiTypography-subtitle1')
        # print(text)
        image = article.find('img')

        if (link != None and date != None and title != None and text != None):
            parsed_date = dateparser.parse(date.get_text())
            companies.append('Geotab')
            description.append(text.get_text().strip())
            publish_date.append(parsed_date.date())
            stories.append('https://www.geotab.com' + link.get('href'))
            titles.append(title.getText().strip())
            
            if (image.get('src')):
                images.append(image.get('src'))
            elif (image.get('data-src')):
                images.append(image.get('data-src'))
            else:
                images.append(None)
                print(article)
            
            company_urls.append('https://www.geotab.com/')

    dataframe = pd.DataFrame({'company':companies, 'headline':titles, 'description':description, 'publish_date':publish_date, 'story_url':stories, 'company_url':company_urls, 'image':images})
    return dataframe

geotab_news = get_geotab_news(geotab_news_content)

geotab_news

,company,headline,description,publish_date,story_url,company_url,image
0,Geotab,Geotab secures its spot on the Globe and Mail'...,Geotab secures its spot on the Globe and Mail'...,2021-09-24,https://www.geotab.com/press-release/top-growi...,https://www.geotab.com/,https://storage.googleapis.com/geotab_wfm_prod...
1,Geotab,Geotab selected as sole telematics provider fo...,Geotab selected as sole telematics provider fo...,2021-09-21,https://www.geotab.com/press-release/oklahoma-...,https://www.geotab.com/,https://storage.googleapis.com/geotab_wfm_prod...
2,Geotab,Geotab Intelligent Transportation Systems (ITS...,Geotab Intelligent Transportation Systems (ITS...,2021-09-14,https://www.geotab.com/press-release/geotab-it...,https://www.geotab.com/,https://storage.googleapis.com/geotab_wfm_prod...
3,Geotab,Geotab prevails in another patent lawsuit,Geotab prevails in another patent lawsuit,2021-09-09,https://www.geotab.com/press-release/stormborn...,https://www.geotab.com/,https://storage.googleapis.com/geotab_wfm_prod...
4,Geotab,A Europe-focused strategy secures Geotab its s...,A Europe-focused strategy secures Geotab its s...,2021-07-29,https://www.geotab.com/press-release/europe-su...,https://www.geotab.com/,https://storage.googleapis.com/geotab_wfm_prod...
5,Geotab,"Geotab Enters Brazilian Market, Bringing Innov...","Geotab Enters Brazilian Market, Bringing Innov...",2021-06-29,https://www.geotab.com/press-release/brazil-ex...,https://www.geotab.com/,https://storage.googleapis.com/geotab_wfm_prod...
6,Geotab,Geotab and Xtract Partner to Bring Advanced In...,Geotab and Xtract Partner to Bring Advanced In...,2021-06-21,https://www.geotab.com/press-release/xtract-pa...,https://www.geotab.com/,https://storage.googleapis.com/geotab_wfm_prod...
7,Geotab,Geotab and Together for Safer Roads join force...,Geotab and Together for Safer Roads join force...,2021-05-10,https://www.geotab.com/press-release/global-ro...,https://www.geotab.com/,https://storage.googleapis.com/geotab_wfm_prod...
8,Geotab,QEV Technologies helps boost the transition to...,QEV Technologies helps boost the transition to...,2021-05-06,https://www.geotab.com/press-release/qev-geotab/,https://www.geotab.com/,https://storage.googleapis.com/geotab_wfm_prod...
9,Geotab,Geotab named one of Canada’s Best Managed Comp...,Geotab named one of Canada’s Best Managed Comp...,2021-05-05,https://www.geotab.com/press-release/best-mana...,https://www.geotab.com/,https://storage.googleapis.com/geotab_wfm_prod...


In [223]:
def get_freightWaves(content):
    soup = BeautifulSoup(content, "html.parser")

    companies = []
    description = []
    publish_date = []
    company_urls = []
    stories = []
    titles = []
    images = []

    articles = soup.find_all(class_='post-item')
    
    for article in articles:
        title = article.find('a')
        date =  article.find(class_='date meta-item tie-icon')
        text =  article.find('p')
        image = article.find('img')

        companies.append('FreightWaves')
        description.append(text.getText())
        publish_date.append(dateparser.parse(date.get_text()).date())
        stories.append(title.get('href'))
        titles.append(title.get('aria-label'))
        images.append(image)
        company_urls.append('https://www.freightwaves.com/')

    dataframe = pd.DataFrame({'company':companies, 'headline':titles, 'description':description, 'publish_date':publish_date, 'story_url':stories, 'company_url':company_urls, 'image':images})
    return dataframe
        
fw_news = get_freightWaves(fw_news_content)

fw_news

,company,headline,description,publish_date,story_url,company_url,image
0,FreightWaves,Getting trucker’s body home logistical nightma...,Tabitha Moshier spent six days getting her hus...,2021-10-01,https://www.freightwaves.com/news/getting-truc...,https://www.freightwaves.com/,[]
1,FreightWaves,Lordstown Motors may sell its plant to Taiwan’...,Taiwan’s Foxconn would manufacture Lordstown M...,2021-09-30,https://www.freightwaves.com/news/lordstown-mo...,https://www.freightwaves.com/,[]
2,FreightWaves,Ransomware attack on Forward Air may have expo...,Nine months after a ransomware attack on Forwa...,2021-09-30,https://www.freightwaves.com/news/ransomware-a...,https://www.freightwaves.com/,[]
3,FreightWaves,Workhorse executive housecleaning continues wi...,"New CEO continues remaking executive team, wit...",2021-09-30,https://www.freightwaves.com/news/workhorse-ex...,https://www.freightwaves.com/,[]
4,FreightWaves,FreightWaves Classics: I-24 travels through fo...,I-24 passes through four states and provides a...,2021-09-30,https://www.freightwaves.com/news/freightwaves...,https://www.freightwaves.com/,[]
5,FreightWaves,M&A activity staying hot across logistics indu...,Pent-up demand is making its way to the surfac...,2021-09-30,https://www.freightwaves.com/news/ma-activity-...,https://www.freightwaves.com/,[]
6,FreightWaves,ArcBest sees billions of dollars in opportunit...,ArcBest’s acquisition of truckload broker MoLo...,2021-09-30,https://www.freightwaves.com/news/arcbest-sees...,https://www.freightwaves.com/,[]
7,FreightWaves,"Be prepared, attorneys say: More driver classi...",A pair of lawyers tell a TCA audience that tak...,2021-09-30,https://www.freightwaves.com/news/be-prepared-...,https://www.freightwaves.com/,[]
8,FreightWaves,Successful driver recruitment is multifaceted ...,Tenstreet partnered with FreightWaves to bette...,2021-09-30,https://www.freightwaves.com/news/successful-d...,https://www.freightwaves.com/,[]
9,FreightWaves,"Pick your bottleneck: Ports, chassis, containe...",With disruptions likely to linger well into 20...,2021-09-30,https://www.freightwaves.com/news/pick-your-bo...,https://www.freightwaves.com/,[]


In [224]:
final = pd.concat([ttnews, fw_news, overdrive_business, samsara_news, platform_science_news, omnitracs_news, geotab_news], axis = 0)
#replace all .coms in company names
final['company'] = final.company.str.replace("[.].*|•|’|,|[+]|[()]|[#]", '').str.strip()

In [225]:
final

,company,headline,description,publish_date,story_url,company_url,image
0,TTNews,Pelosi Regroups on Infrastructure With Hopes f...,House Speaker Nancy Pelosi (D-Calif.) plans to...,2021-10-01,https://ttnews.com/articles/pelosi-regroups-in...,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
1,TTNews,Daimler Investors Approve Plan to Split Merced...,Daimler AG shareholders backed a plan to spin ...,2021-10-01,https://ttnews.com/articles/daimler-investors-...,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
2,TTNews,"Nikola, Opal Fuels to Build Hydrogen Fueling S...",Nikola Corp. reached a preliminary deal with O...,2021-10-01,https://ttnews.com/articles/nikola-opal-fuels-...,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
3,TTNews,"No Vote Held on Infrastructure, but Talks to R...",WASHINGTON — Despite a long night of frantic n...,2021-10-01,https://ttnews.com/articles/no-vote-held-infra...,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
4,TTNews,TT Newsmakers,"Sgt. Zach Heard, Illinois State Police, and Ke...",2021-09-30,https://ttnews.com/newsmakers,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
...,...,...,...,...,...,...,...
9,Geotab,Geotab named one of Canada’s Best Managed Comp...,Geotab named one of Canada’s Best Managed Comp...,2021-05-05,https://www.geotab.com/press-release/best-mana...,https://www.geotab.com/,https://storage.googleapis.com/geotab_wfm_prod...
10,Geotab,Geotab solidifies position as top telematics p...,Geotab solidifies position as top telematics p...,2021-04-27,https://www.geotab.com/press-release/2000-gove...,https://www.geotab.com/,https://storage.googleapis.com/geotab_wfm_prod...
11,Geotab,Geotab appoints Christoph Ludewig as new Vice ...,Geotab appoints Christoph Ludewig as new Vice ...,2021-03-25,https://www.geotab.com/press-release/ludewig-o...,https://www.geotab.com/,https://storage.googleapis.com/geotab_wfm_prod...
12,Geotab,Electric Last Mile and Geotab Inc. announce pa...,Electric Last Mile and Geotab Inc. announce pa...,2021-03-24,https://www.geotab.com/press-release/elms-part...,https://www.geotab.com/,https://storage.googleapis.com/geotab_wfm_prod...


In [226]:
final.head()

,company,headline,description,publish_date,story_url,company_url,image
0,TTNews,Pelosi Regroups on Infrastructure With Hopes f...,House Speaker Nancy Pelosi (D-Calif.) plans to...,2021-10-01,https://ttnews.com/articles/pelosi-regroups-in...,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
1,TTNews,Daimler Investors Approve Plan to Split Merced...,Daimler AG shareholders backed a plan to spin ...,2021-10-01,https://ttnews.com/articles/daimler-investors-...,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
2,TTNews,"Nikola, Opal Fuels to Build Hydrogen Fueling S...",Nikola Corp. reached a preliminary deal with O...,2021-10-01,https://ttnews.com/articles/nikola-opal-fuels-...,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
3,TTNews,"No Vote Held on Infrastructure, but Talks to R...",WASHINGTON — Despite a long night of frantic n...,2021-10-01,https://ttnews.com/articles/no-vote-held-infra...,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
4,TTNews,TT Newsmakers,"Sgt. Zach Heard, Illinois State Police, and Ke...",2021-09-30,https://ttnews.com/newsmakers,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...


In [227]:
len(final)

270

In [228]:
#check previous posts, check to see there are no duplicates
# old = pd.read_csv('historical_deals.csv')
# dupes = pd.merge(final, old[['company', 'deal_size']], how = 'inner', on = ['company', 'deal_size'])
# final = final[~final.company.isin(dupes.company)]

In [229]:
len(final)

270

In [230]:
final

,company,headline,description,publish_date,story_url,company_url,image
0,TTNews,Pelosi Regroups on Infrastructure With Hopes f...,House Speaker Nancy Pelosi (D-Calif.) plans to...,2021-10-01,https://ttnews.com/articles/pelosi-regroups-in...,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
1,TTNews,Daimler Investors Approve Plan to Split Merced...,Daimler AG shareholders backed a plan to spin ...,2021-10-01,https://ttnews.com/articles/daimler-investors-...,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
2,TTNews,"Nikola, Opal Fuels to Build Hydrogen Fueling S...",Nikola Corp. reached a preliminary deal with O...,2021-10-01,https://ttnews.com/articles/nikola-opal-fuels-...,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
3,TTNews,"No Vote Held on Infrastructure, but Talks to R...",WASHINGTON — Despite a long night of frantic n...,2021-10-01,https://ttnews.com/articles/no-vote-held-infra...,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
4,TTNews,TT Newsmakers,"Sgt. Zach Heard, Illinois State Police, and Ke...",2021-09-30,https://ttnews.com/newsmakers,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
...,...,...,...,...,...,...,...
9,Geotab,Geotab named one of Canada’s Best Managed Comp...,Geotab named one of Canada’s Best Managed Comp...,2021-05-05,https://www.geotab.com/press-release/best-mana...,https://www.geotab.com/,https://storage.googleapis.com/geotab_wfm_prod...
10,Geotab,Geotab solidifies position as top telematics p...,Geotab solidifies position as top telematics p...,2021-04-27,https://www.geotab.com/press-release/2000-gove...,https://www.geotab.com/,https://storage.googleapis.com/geotab_wfm_prod...
11,Geotab,Geotab appoints Christoph Ludewig as new Vice ...,Geotab appoints Christoph Ludewig as new Vice ...,2021-03-25,https://www.geotab.com/press-release/ludewig-o...,https://www.geotab.com/,https://storage.googleapis.com/geotab_wfm_prod...
12,Geotab,Electric Last Mile and Geotab Inc. announce pa...,Electric Last Mile and Geotab Inc. announce pa...,2021-03-24,https://www.geotab.com/press-release/elms-part...,https://www.geotab.com/,https://storage.googleapis.com/geotab_wfm_prod...


In [231]:
#save deal records
# new_old = pd.concat([final, old], axis = 0)
new_old = pd.concat([final], axis = 0)

new_old.to_csv('historical_deals.csv', index = False, sep = ';')

In [232]:
len(new_old)

270

In [233]:
date = 'Sunday, 08/29/2021'

title = '''
<h1>
  Funded & Hiring
</h1>
<h2 style="text-align:center">
  <strong>{date}</strong>
</h2>
<h2>
  A weekly roundup of funded startups and the jobs they're hiring for.
</h2>
<h4>
  Forwarded from a friend? 
  <a href = 'https://fundedandhiring.com/'>Subscribe</a> to stay up to date on the latest startup funding and job alerts.
</h4>
'''.format(date = date).strip()

In [234]:
print(title)
pyperclip.copy(title)

<h1>
  Funded & Hiring
</h1>
<h2 style="text-align:center">
  <strong>Sunday, 08/29/2021</strong>
</h2>
<h2>
  A weekly roundup of funded startups and the jobs they're hiring for.
</h2>
<h4>
  Forwarded from a friend? 
  <a href = 'https://fundedandhiring.com/'>Subscribe</a> to stay up to date on the latest startup funding and job alerts.
</h4>


In [235]:
content = ''''''

final_dict = final.to_dict(orient = 'records')
final_dict = final_dict[0:7]
for row in final_dict:
    headline = row['headline']
    company_name = row['company']
    publish_date = row['publish_date']
    description = row['description']
    story_url = row['story_url']
    company_url = row['company_url']
    img = row['image']

    row_url = company_url + story_url
    
    entry = '''
    <tr>
      <td>
        <image src="{img}"/>
        <h3>
          <a href = "{row_url}">{headline}</a>
        </h3>
        <p>
          {description}
        </p>
        <p>
          <a href = "{company_url}">{company_name}</a>
        </p>
        <p>
          <b>{publish_date}</b>
        </p>
      </td>
    </tr> '''.format(row_url = row_url, headline = headline, company_url = company_url, company_name = company_name, description = description, publish_date = publish_date, img=img).strip()

    content += '\n' + entry.strip()

In [236]:
pyperclip.copy(content)
print(content)


<tr>
      <td>
        <image src="https://h7f7z2r7.stackpathcdn.com/sites/default/files/styles/image-528-293/public/images/articles/infra-manchin-ap-1200.jpg?itok=2cLgDdOv"/>
        <h3>
          <a href = "https://ttnews.com/https://ttnews.com/articles/pelosi-regroups-infrastructure-hopes-vote-today">Pelosi Regroups on Infrastructure With Hopes for Vote Today</a>
        </h3>
        <p>
          House Speaker Nancy Pelosi (D-Calif.) plans to try again Oct. 1 for a vote on bipartisan infrastructure legislation that has been held up by a battle between moderate and progressive Democrats over President Joe Biden’s economic agenda.
        </p>
        <p>
          <a href = "https://ttnews.com/">TTNews</a>
        </p>
        <p>
          <b>2021-10-01</b>
        </p>
      </td>
    </tr>
<tr>
      <td>
        <image src="https://h7f7z2r7.stackpathcdn.com/sites/default/files/styles/image-528-293/public/images/articles/daimler-kallenius-1200_0.jpg?itok=Yc_OczDZ"/>
        <

In [237]:
#job boards
# stackoverflow, jobvite, bamboohr